<a href="https://colab.research.google.com/github/AI-Maker-Space/LLM-Ops-Vault/blob/main/Week%201/First%20Session/Barbie_Retrieval_Augmented_Question_Answering_(RAQA)_Assignment%20(Assignment%20Version).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Twitter with RAQA (Retrieval Augmented Question Answering)

In the following notebook, you are tasked with creating a system that answers questions based on information found in tweets on Depression / ADHD
## Build 🏗️

There are 3 main tasks in this notebook:

1. Obtain and parse reviews from a review website
2. Create a Vectorstore from the reviews
3. Create a `RetrievalQA` using the VectorStore

## Ship 🚢

Create a Hugging Face Space that hosts your application.

## Share 🚀

Make a social media post about your final application.

>### Why RAQA and not RAG?
>If we look at the original [paper](https://arxiv.org/abs/2005.11401), we find that RAG is a fairly specific and well defined term that isn't exactly the same as "retrieve context, feed context to model in the prompt".
>For that reason, we're making the decision to delineate between "actual" RAG, and Retrieval Augmented Question Answering - which is not a well defined phrase.

### Pre-task Work

All we really need to do to get started is to get our prerequisites!

We'll be leveraging `langchain` and `openai` today.

Check out the docs:
- [LangChain](https://docs.langchain.com/docs/)
- [OpenAI](https://github.com/openai/openai-python)

In [8]:
!pip install -q -U openai langchain

In [24]:
import os
import getpass

openai_api_key = getpass.getpass("Enter your OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

### Task 1: Data Preparation

In this task we'll be collecting, and then parsing, our data.

#### Scraping IMDB Reviews of Barbie

We'll use some Selenium based trickery to get the reviews we need to make our application.

Check out the docs here:
- [Selenium](https://www.selenium.dev/documentation/)

In [2]:
!pip install -q -U requests pandas 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index 0.8.6 requires langchain<=0.0.266,>=0.0.262, but you have langchain 0.0.281 which is incompatible.


In [3]:
!pip install -U ipykernel

  Obtaining dependency information for ipykernel from https://files.pythonhosted.org/packages/94/e3/70fb6e6bdd42cb0586d6b6680713997d2a9dc46642aec207ca04d0df80b8/ipykernel-6.25.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.2/154.2 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 6.25.1
    Uninstalling ipykernel-6.25.1:
      Successfully uninstalled ipykernel-6.25.1


In [12]:
!pip install -q -U scrapy selenium

In [13]:
!apt install chromium-chromedriver 

The operation couldn’t be completed. Unable to locate a Java Runtime that supports apt.
Please visit http://www.java.com for information on installing Java.



In [4]:
import numpy as np
import pandas as pd
from scrapy.selector import Selector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [5]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

In [16]:
#url = "https://www.imdb.com/title/tt1517268/reviews/?ref_=tt_ov_rt"
#driver.get(url)

In [17]:
#sel = Selector(text = driver.page_source)
#review_counts = sel.css('.lister .header span::text').extract_first().replace(',','').split(' ')[0]
#more_review_pages = int(int(review_counts)/25)

In [12]:
review_df

,Review_Date,Author,Rating,Review_Title,Review,Review_Url
0,21 July 2023,LoveofLegacy,6,"Beautiful film, but so preachy\n","Margot does the best with what she's given, bu...",/review/rw9201978/?ref_=tt_urv
1,22 July 2023,imseeg,7,3 reasons FOR seeing it and 1 reason AGAINST.\n,The first reason to go see it:,/review/rw9201978/?ref_=tt_urv
2,22 July 2023,Natcat87,6,Too heavy handed\n,"As a woman that grew up with Barbie, I was ver...",/review/rw9201978/?ref_=tt_urv
3,31 July 2023,ramair350,10,"As a guy I felt some discomfort, and that's o...",As much as it pains me to give a movie called ...,/review/rw9201978/?ref_=tt_urv
4,24 July 2023,heatherhilgers,9,A Technicolor Dream\n,"Wow, this movie was a love letter to cinema. F...",/review/rw9201978/?ref_=tt_urv
...,...,...,...,...,...,...
70,28 July 2023,klastaitas,8,We're All Dolls\n,"Without consumerism, without belief, without p...",/review/rw9201978/?ref_=tt_urv
71,20 July 2023,FeastMode,7,I'll beach you off so hard\n,I had no idea what to expect going into Barbie...,/review/rw9201978/?ref_=tt_urv
72,7 August 2023,misscattik,None,No questions just answers\n,The title of the review practically sums it al...,/review/rw9201978/?ref_=tt_urv
73,23 July 2023,acdc_mp3,9,A really fun time\n,Weeks ago I heard that there was going to be a...,/review/rw9201978/?ref_=tt_urv


Let's save this `pd.DataFrame` as a `.csv` to our local session (this will be terminated when you terminate the Colab session) so we can leverage it in LangChain!

Check out the docs if you get stuck:
- [`to_csv()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html)

In [13]:
### YOUR CODE HERE
review_df.to_csv("./movie_review.csv")

In [8]:
adhd_df = pd.read_csv('adhd-twitter.csv')
adhd_df


,fullname,id,images/0,isQuote,isRetweet,likes,quotes,replies,retweets,text,timestamp,url,username,verified
0,★ skye/nate AKO DAY,1698458321026465884,https://pbs.twimg.com/media/F5Ii10OXsAA3zqU.jpg,False,False,0,0,0,0,i’m getting verified for ADHD starting TOMORRO...,2023-09-03T22:09:00.000Z,https://twitter.com/skyelights__/status/169845...,@skyelights__,False
1,Ellie R. Tran (thewriterlydragonslayer),1698458318602195060,NaN,False,False,0,0,1,0,I can’t even tell CUTE stories about Eve walki...,2023-09-03T22:09:00.000Z,https://twitter.com/writerly_ellie/status/1698...,@writerly_ellie,False
2,Taita Tia Trinity Trismegistus (Mother),1698458311136256289,NaN,False,False,0,0,0,0,ADHD is scientifically proven to be the result...,2023-09-03T22:09:00.000Z,https://twitter.com/jenymcg1898/status/1698458...,@jenymcg1898,False
3,Console Pit,1698458255918285107,NaN,False,False,0,0,0,0,I really struggle with hydrating. Literally wa...,2023-09-03T22:09:00.000Z,https://twitter.com/ConsolePit/status/16984582...,@ConsolePit,False
4,Azalea,1698458176348127743,NaN,False,False,0,0,0,0,"Damn, 1 hour of sleep -> forgetting to take AD...",2023-09-03T22:09:00.000Z,https://twitter.com/azalea_bu/status/169845817...,@azalea_bu,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,myADHDlife,1698384244446556232,NaN,False,False,0,0,0,0,ADHD-CAN YOU TURN THAT DOWN?!!,2023-09-03T17:15:00.000Z,https://twitter.com/midlifeADHD/status/1698384...,@midlifeADHD,False
996,GentleBear,1698384220324839453,NaN,False,False,0,0,0,0,As you yourself might know. School is a format...,2023-09-03T17:15:00.000Z,https://twitter.com/meapopuli/status/169838422...,@meapopuli,False
997,devinendorphin,1698384136338079921,NaN,False,False,0,0,1,0,Had similar exp but in Chemsex space. Lost job...,2023-09-03T17:15:00.000Z,https://twitter.com/glubose/status/16983841363...,@glubose,False
998,ADHDDude,1698384048266068130,NaN,False,False,0,0,0,0,By the way my question was rhetorical. I suspe...,2023-09-03T17:14:00.000Z,https://twitter.com/DudeAdhd/status/1698384048...,@DudeAdhd,False


In [59]:
depression_df = pd.read_csv('Mental-Health-Twitter 3.csv')
depression_df


,Unnamed: 0,post_id,post_created,post_text,user_id,followers,friends,favourites,statuses,retweets,label
0,0,637894677824413696,Sun Aug 30 07:48:37 +0000 2015,It's just over 2 years since I was diagnosed w...,1013187241,84,211,251,837,0,1
1,1,637890384576778240,Sun Aug 30 07:31:33 +0000 2015,"It's Sunday, I need a break, so I'm planning t...",1013187241,84,211,251,837,1,1
2,2,637749345908051968,Sat Aug 29 22:11:07 +0000 2015,Awake but tired. I need to sleep but my brain ...,1013187241,84,211,251,837,0,1
3,3,637696421077123073,Sat Aug 29 18:40:49 +0000 2015,RT @SewHQ: #Retro bears make perfect gifts and...,1013187241,84,211,251,837,2,1
4,4,637696327485366272,Sat Aug 29 18:40:26 +0000 2015,It’s hard to say whether packing lists are mak...,1013187241,84,211,251,837,1,1
...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,819336825231773698,Thu Jan 12 00:14:56 +0000 2017,A day without sunshine is like night.,1169875706,442,230,7,1063601,0,0
19996,19996,819334654260080640,Thu Jan 12 00:06:18 +0000 2017,"Boren's Laws: (1) When in charge, ponder. (2) ...",1169875706,442,230,7,1063601,0,0
19997,19997,819334503042871297,Thu Jan 12 00:05:42 +0000 2017,The flow chart is a most thoroughly oversold p...,1169875706,442,230,7,1063601,0,0
19998,19998,819334419374899200,Thu Jan 12 00:05:22 +0000 2017,"Ships are safe in harbor, but they were never ...",1169875706,442,230,7,1063601,0,0


In [62]:
depression_df=depression_df[:300]
len(depression_df)

300

In [63]:
depression_df.to_csv('depression_df.csv') 

In [10]:
list(adhd_df.columns.values)

['fullname',
 'id',
 'images/0',
 'isQuote',
 'isRetweet',
 'likes',
 'quotes',
 'replies',
 'retweets',
 'text',
 'timestamp',
 'url',
 'username',
 'verified']

#### Data Parsing

Now that we have our data - let's go ahead and start parsing it into a more usable format for LangChain!

We'll be using the `CSVLoader` for this application.

We also want to be sure to track the sources that our review came from!

Check out the docs here:
- [`CSVLoader`](https://python.langchain.com/docs/integrations/document_loaders/csv)

In [32]:
from langchain.document_loaders.csv_loader import CSVLoader

In [33]:
loader = CSVLoader(
    file_path="adhd-twitter.csv",
    source_column='url'
    )
 
data = loader.load()

In [27]:
#print(data)

In [35]:
#assert len(data) == 125
len(data)

1000

In [13]:
len(data)

1000

Now that we have collected our review information into a loader - we can go ahead and chunk the reviews into more manageable pieces.

We'll be leveraging the `RecursiveCharacterTextSplitter` for this task today.

While splitting our text seems like a simple enough task - getting this correct/incorrect can have massive downstream impacts on your application's performance.

You can read the docs here:
- [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter)

We want to split our documents into 1000 character length chunks, with 100 characters of overlap.

> ### HINT:
>It's always worth it to check out the LangChain source code if you're ever in a bind - for instance, if you want to know how to transform a set of documents, check it out [here](https://github.com/langchain-ai/langchain/blob/5e9687a196410e9f41ebcd11eb3f2ca13925545b/libs/langchain/langchain/text_splitter.py#L268C18-L268C18)

In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, # the character length of the chunk
    chunk_overlap = 100, # the character length of the overlap between chunks
    length_function = len # the length function
)

In [37]:
documents = text_splitter.transform_documents(data)### YOUR CODE HERE

In [28]:
#print(documents)

In [38]:
#assert len(documents) == 181
len(documents)

1004

With our documents transformed into more manageable sizes, and with the correct metadata set-up, we're now ready to move on to creating our VectorStore!

### Task 2: Creating an "Index"

The term "index" is used largely to mean: Structured documents parsed into a useful format for querying, retrieving, and use in the LLM application stack.

#### Selecting Our VectorStore

There are a number of different VectorStores, and a number of different strengths and weaknesses to each.

In this notebook, we will be keeping it very simple by leveraging [Facebook AI Similarity Search](https://ai.meta.com/tools/faiss/#:~:text=FAISS%20(Facebook%20AI%20Similarity%20Search,more%20scalable%20similarity%20search%20functions.), or `FAISS`.

In [22]:
!pip install -q -U faiss-cpu tiktoken

We're going to be setting up our VectorStore with the OpenAI embeddings model. While this embeddings model does not need to be consistent with the LLM selection, it does need to be consistent between embedding our index and embedding our queries over that index.

While we don't have to worry too much about that in this example - it's something to keep in mind for more complex applications.

We're going to leverage a [`CacheBackedEmbeddings`](https://python.langchain.com/docs/modules/data_connection/caching_embeddings )flow to prevent us from re-embedding similar queries over and over again.

Not only will this save time, it will also save us precious embedding tokens, which will reduce the overall cost for our application.

>#### Note:
>The overall cost savings needs to be compared against the additional cost of storing the cached embeddings for a true cost/benefit analysis. If your users are submitting the same queries often, though, this pattern can be a massive reduction in cost.

In [39]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/") ### YOUR CODE HERE ??
core_embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key) ### YOUR CODE HERE

embedder = CacheBackedEmbeddings.from_bytes_store(
core_embeddings_model, store, namespace=core_embeddings_model.model
)

vector_store =FAISS.from_documents(documents,embedder)### YOUR CODE HERE

In [41]:
list(store.yield_keys())

['text-embedding-ada-002c99c9897-7e53-5ae6-8f30-799a1978ec74',
 'text-embedding-ada-002c0b2e6b0-677b-53d0-b091-4aeeb891729a',
 'text-embedding-ada-002089fe89f-d8ca-53d1-b619-91514d4c6b34',
 'text-embedding-ada-002cf9e8a50-4d64-559e-a67a-d387ff0e5672',
 'text-embedding-ada-002e6378de5-595f-5cac-9fe8-ddb67a7db2fb',
 'text-embedding-ada-0020603c63f-24dd-5a49-af5a-3387f8de2474',
 'text-embedding-ada-002e0e41d83-5c0f-52d4-92c1-a5f7c0e817a1',
 'text-embedding-ada-002debccc49-d903-5908-9eae-064b18d14083',
 'text-embedding-ada-0024b78875d-3ffb-5aa3-8ea9-34cd36d63493',
 'text-embedding-ada-002ecf91fa6-b821-58a8-8e3a-acc20824ca6d',
 'text-embedding-ada-002b7e8f774-5c15-57bb-a1f8-3effaed6afee',
 'text-embedding-ada-0026f0407c9-e496-579c-a04e-ef6f1b40543a',
 'text-embedding-ada-002e2c67c36-d9c7-5279-af35-1f8dfbe2e40e',
 'text-embedding-ada-00259d209da-619b-5fd3-9160-cc403cd119da',
 'text-embedding-ada-002b1568535-76df-51ce-a5cc-4233ff229453',
 'text-embedding-ada-00240969db1-1687-5fb2-9ad1-71b8b52

Now that we've created the VectorStore, we can check that it's working by embedding a query and retrieving passages from our reviews that are close to it.

In [42]:
query = "What is school like for people with ADHD?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)

"fullname": Spvwvky 👻 VTuber | Music & Motion Graphics
id: 1698451270162403837
images/0: 
isQuote: false
isRetweet: false
likes: 0
quotes: 0
replies: 0
retweets: 0
text: It's hard to put it into one word tbh. It'd probably be more effective to send someone resources that talk about ADHD/Autistic memory skills and how it affects us on a personal level if your intention is to get them to be more understanding (and they are willing to check it out) - plus the nice bonus is that published sources are typically a lot better at articulating these concepts than we are since we often use emotions and how we feel to explain what things are like. Luckily these days you can find tons of useful resources with a single Google search to help people learn about neurodivergence which I think really lends to the growing number of people that exist online that are more aware of the presence of ND people around them.
﻿"fullname": unfettered by neuronormativity
id: 1698429799126225390
images/0: 
isQuote: 

Let's see how much time the `CacheBackedEmbeddings` pattern saves us:

In [43]:
%%timeit -n 1 -r 1
query = "My son has adhd, and I worry if going to school is going to be difficult for him."
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

225 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [44]:
#%%timeit
query = "My son has adhd, and I worry if going to school is going to be difficult for him."
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

In [46]:
#%%timeit
query = "My daughter has adhd, and I worry if going to school is going to be difficult for her."
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

In [47]:
for page in docs:
  print(page.page_content)

"fullname": Julia Gordon-Bramer
id: 1698396223248412916
images/0: 
isQuote: false
isRetweet: false
likes: 11
quotes: 0
replies: 0
retweets: 0
text: If I'd been born in an earlier generation, I'm sure I would be labeled ADHD. I think it's a gift. I'm highly creative, disciplined if I can make the rules, intensely focused even if I change tracks quickly, never bored, always switching to something or connecting things in new ways. I think these kids need to quit labeling themselves with problems and just find a way to work with their own patterns.
timestamp: 2023-09-03T18:03:00.000Z
url: https://twitter.com/JGordonBramer/status/1698396223248412916
username: @JGordonBramer
verified: true
﻿"fullname": Spvwvky 👻 VTuber | Music & Motion Graphics
id: 1698451270162403837
images/0: 
isQuote: false
isRetweet: false
likes: 0
quotes: 0
replies: 0
retweets: 0
text: It's hard to put it into one word tbh. It'd probably be more effective to send someone resources that talk about ADHD/Autistic memory sk

As we can see, even over a significant number of runs - the cached query is significantly faster than the first instance of the query!

With that, we're ready to move onto Task 3!

### Task 3: Building a Retrieval Chain

In this task, we'll be making a Retrieval Chain which will allow us to ask semantic questions over our data.

This part is rather abstracted away from us in LangChain and so it seems very powerful.

Be sure to check the documentation, the source code, and other provided resources to build a deeper understanding of what's happening "under the hood"!

#### A Basic RetrievalQA Chain

We're going to leverage `return_source_documents=True` to ensure we have proper sources for our reviews - should the end user want to verify the reviews themselves.

Hallucinations [are](https://arxiv.org/abs/2202.03629) [a](https://arxiv.org/abs/2305.15852) [massive](https://arxiv.org/abs/2303.16104) [problem](https://arxiv.org/abs/2305.18248) in LLM applications.

Though it has been tenuously shown that using Retrieval Augmentation [reduces hallucination in conversations](https://arxiv.org/pdf/2104.07567.pdf), one sure fire way to ensure your model is not hallucinating in a non-transparent way is to provide sources with your responses. This way the end-user can verify the output.

#### Our LLM

In this notebook, we'll continue to leverage OpenAI's suite of models - this time we'll be using the `gpt-3.5-turbo` model to power our RetrievalQAWithSources chain.

Check out the relevant documentation if you get stuck:
- [`OpenAIChat()`](https://python.langchain.com/docs/modules/model_io/models/chat/)

In [48]:
from langchain.llms.openai import OpenAIChat

# llm = ### YOUR CODE HERE

In [49]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo"),
#chat = ChatOpenAI(model_name="gpt-3.5-turbo")

Now we can set up our chain.

We'll need to make our VectorStore a retriever in order to be able to leverage it in our chain - let's check out the `as_retriever()` method.

Relevant Documentation:
- [`FAISS`](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.faiss.FAISS.html#langchain.vectorstores.faiss.FAISS.as_retriever)

In [50]:
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 3, 'fetch_k': 50}
)

# retriever = vector_store.as_retriever(
#     search_type="mmr",
#     search_kwargs={'k': 6, 'lambda_mult': 0.25}
# )

In [51]:
#https://python.langchain.com/docs/use_cases/question_answering/how_to/vector_db_qa
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()

qa_with_sources_chain = RetrievalQA.from_chain_type(
    #chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, streaming=True),
    llm= ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, streaming=True),### YOUR CODE HERE,
    retriever=retriever,### YOUR CODE HERE,
    callbacks=[handler],
    return_source_documents=True
)

In [54]:
qa_with_sources_chain({"query" : "How do peopel with ADHD view adderal?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'How do peopel with ADHD view adderal?',
 'result': 'Based on the provided context, we can see that people with ADHD have different views on Adderall. The user "trannygirlsex" expresses gratitude for Adderall, suggesting that they may find it helpful in treating their ADHD. However, it\'s important to note that this is just one individual\'s perspective. Other individuals with ADHD may have different opinions or experiences with Adderall.',
 'source_documents': [Document(page_content='\ufeff"fullname": yeezypoleon Thank God for Drugs 🐻💽\nid: 1698393192574411180\nimages/0: \nisQuote: false\nisRetweet: false\nlikes: 1\nquotes: 0\nreplies: 2\nretweets: 0\ntext: Adderall literally treats ADHD😭😭\ntimestamp: 2023-09-03T17:51:00.000Z\nurl: https://twitter.com/trannygirlsex/status/1698393192574411180\nusername: @trannygirlsex\nverified: false', metadata={'source': 'https://twitter.com/trannygirlsex/status/1698393192574411180', 'row': 833}),
  Document(page_content='\ufeff"fullname": 

In [55]:
qa_with_sources_chain({"query" : "what is a major sympton of ADHD ?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'what is a major sympton of ADHD ?',
 'result': 'A major symptom of ADHD is difficulty with attention and focus. People with ADHD often struggle to stay on task, get easily distracted, and have trouble organizing and completing tasks.',
 'source_documents': [Document(page_content='\ufeff"fullname": Frif 🟪\nid: 1698433179252560372\nimages/0: \nisQuote: false\nisRetweet: false\nlikes: 20\nquotes: 0\nreplies: 1\nretweets: 2\ntext: If you perform extremely well on major grades like tests and projects, but daily work and especially homework brings your grades down, you probably have ADHD.\n\nIf you\'re able to, talk to a doctor about it. Living without some kind of treatment for that is like hard mode.\ntimestamp: 2023-09-03T20:29:00.000Z\nurl: https://twitter.com/Frifuloid/status/1698433179252560372\nusername: @Frifuloid\nverified: false', metadata={'source': 'https://twitter.com/Frifuloid/status/1698433179252560372', 'row': 276}),
  Document(page_content='\ufeff"fullname": Spvwv

In [58]:
qa_with_sources_chain({"query" : "does women with adhd has experience different from man?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'does women with adhd has experience different from man?',
 'result': 'Yes, women with ADHD can have different experiences compared to men with ADHD. The symptoms and presentation of ADHD can vary between genders. It may be helpful to look up symptoms specifically in women, as they can differ from those typically associated with men. Understanding these differences can provide valuable insights and support for women with ADHD.',
 'source_documents': [Document(page_content='\ufeff"fullname": lizzy\nid: 1698435620023017861\nimages/0: \nisQuote: false\nisRetweet: false\nlikes: 1\nquotes: 0\nreplies: 0\nretweets: 0\ntext: there’s like different types of adhd like type?? and it will definitely be helpful to look up symptoms in WOMEN bc it changes a lot from guys, a big thing for me is off my meds i’m annoying?? like i’ve always like had that made apparent to me but def look at the symptoms\ntimestamp: 2023-09-03T20:39:00.000Z\nurl: https://twitter.com/lamictalspo/status/1698435620

And with that, we have our Barbie Review RAQA Application built!

### Conclusion

Now that we have our application ready, let's deploy it to a Hugging Face space with their Docker spaces.

You can find the next part [here](https://ai-maker-space-barbie-raqa-application-chainlit-demo.hf.space/readme)!

You've built, now it's time to ship! 🚢